<a href="https://colab.research.google.com/github/Syed-Raza-Ali/Browsing_AI_Agent/blob/main/Browsing_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Tavily-Enhanced Real-Time Browsing Agent**

- **A dynamic browsing agent leveraging LangGraph and the Tavily real-time search API to provide instant, accurate responses to user queries. Powered by the Gemini-1.5-Flash LLM and integrated tools, the agent retrieves, processes, and delivers relevant information in a user-friendly format. Designed for seamless and efficient communication, this agent ensures users get the insights they need without technical jargon or raw data, making it ideal for real-time information access.**

In [ ]:
# install the required packages
%%capture --no-stderr
%pip install -U tavily-python langchain_core langchain_google_genai tavily-python langchain_community langgraph python-dotenv

In [ ]:
# API KEYS VARIABLES
import os
from google.colab import userdata

os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")
GOOGLE_API_KEY = userdata.get("GEMINI_API_KEY")


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import MessagesState
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition
from langgraph.prebuilt import ToolNode
from IPython.display import Image, display
from langgraph.graph.state import CompiledStateGraph


def main_three():
  """ Get realtime information using tavily search """

  while True:


    try:
        # Prompt the user for input
        print("\n")
        user_input = input("User : ").strip()

        # Exit condition
        if user_input.lower() in ["quit", "exit", "q","bye"]:
            print("Bye! Have a great day!")
            break

        # Handle empty input
        if not user_input:
            print("No prompt found. Please type something.")
            continue

        # Add user input to chat history
        chat_history.append({"role": "user", "content": user_input})

        # Call the placeholder function (replace as needed)
        try:
            stream_graph_updates(user_input)
        except NameError:
            # Handle missing function gracefully
            print("stream_graph_updates is not implemented. Skipping.")

        # Perform a Tavily search if the function is not implemented
        search_docs = tavily_search.invoke()

        # Add Tavily response to chat history
        chat_history.append({"role": "assistant", "content": str(search_docs)})

        # Display results from Tavily search
        print("Search Results:")
        for i, doc in enumerate(search_docs, start=1):
            print(f"\nResult {i}:")
            print(f"Here is the {i} documentaion from web search you can get your answer from here:")
            print(f"Documentation: {doc.get('content', 'N/A')}")
            print(f"URL: {doc.get('url', 'N/A')}")

    except Exception as e:
        print(f"An error occurred: {e}")
        break

    print("\nChat History:")
    for message in chat_history:
      role = message["role"]
      content = message["content"]
      print(f"{role.capitalize()}: {content}")

tools = [main_three]
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", api_key = GOOGLE_API_KEY)
llm_with_tools = llm.bind_tools(tools)

# System message
sys_msg = SystemMessage(content="You are a helpful assistant tasked with performing realtime searching using tavily ")

# Node
def assistant(state: MessagesState) -> MessagesState:
   return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}


# Graph
builder: StateGraph = StateGraph(MessagesState)

# Define nodes: these do the work
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Define edges: these determine how the control flow moves
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    # If the latest message (result) from assistant is a tool call -> tools_condition routes to tools
    # If the latest message (result) from assistant is a not a tool call -> tools_condition routes to END
    tools_condition,
)
builder.add_edge("tools", "assistant")
react_graph: CompiledStateGraph = builder.compile()

# Show
# display(Image(react_graph.get_graph(xray=True).draw_mermaid_png()))

messages = [HumanMessage(content="hey can you tell me who is the prime minister of pakistan in 2020")]
messages = react_graph.invoke({"messages": messages})
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

hey can you tell me who is the prime minister of pakistan in 202
================================== Ai Message ==================================

I am sorry, I cannot answer this question. The available tools lack the functionality to access information about the prime minister of Pakistan in a specific year.


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import MessagesState
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition
from langgraph.prebuilt import ToolNode
from IPython.display import Image, display
from langgraph.graph.state import CompiledStateGraph
from langchain.agents import Tool

# Correctly define the tool in a format that LangChain expects
def realtime_search(query: str) -> str:
    """
    Function to perform real-time searching using Tavily.
    """
    from tavily import TavilyClient  # Ensure TavilyClient is installed and configured
    client = TavilyClient()  # Replace with your API key
    try:
        results = client.search(query)
        title = results["results"][0]["title"]
        # print(f"tittle:{title}")
        return f"Search Query: {title}"  # Format the results as a string
    except Exception as e:
        return f"Error during search: {str(e)}"

# Define the tool correctly as a callable object
realtime_search_tool = Tool(
    name="realtime_search",
    func=realtime_search,
    description="This tool performs real-time searches using Tavily and returns relevant results."
)


# Add tools to the tools list
tools = [realtime_search_tool]

# Initialize the LLM
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", api_key=GOOGLE_API_KEY)
llm_with_tools = llm.bind_tools(tools)

# Define system message
sys_msg = (
    "you are a realtime searching llm you are expert to use your tool for realtime searching when user asked anything You can use this tool to search and give response to me make sure dont give object to user give response in readable form:\n"
    "1. realtime_search: Search for real-time information on the web.\n"
)

# Define the assistant function
def assistant(state: MessagesState) -> MessagesState:
    """
    Assistant function that handles LLM responses.
    """
    return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}

# Create the graph
builder: StateGraph = StateGraph(MessagesState)

# Add nodes
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Define edges
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    tools_condition,  # If the assistant response is a tool call
)
builder.add_edge("tools", "assistant")

# Compile the graph
react_graph: CompiledStateGraph = builder.compile()

# User interaction
messages = [HumanMessage(content=input("User: "))]
messages = react_graph.invoke({"messages": messages})

# Print the output messages
for m in messages['messages']:
    print(m.content)





User: hey can you tell me who is the prime minister of pakistan in 2024
hey can you tell me who is the prime minister of pakistan in 2024
I cannot provide the exact answer to who the Prime Minister of Pakistan will be in 2024.  Predicting future political events with certainty is impossible.  Real-time search tools can only access currently available information.  To find out who the Prime Minister is at that time, you will need to check reputable news sources in 2024.


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import MessagesState
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition
from langgraph.prebuilt import ToolNode
from langgraph.graph.state import CompiledStateGraph
from langchain.agents import Tool



# Tavily real-time search function
def realtime_search(query: str) -> str:
    """
    Function to perform real-time searching using Tavily.
    """
    from tavily import TavilyClient  # Ensure TavilyClient is installed and configured
    client = TavilyClient()  # Replace with your API key
    try:
        results = client.search(query)
        # title = query["results"][0]["title"]

        return f"Search Results: {results}"  # Format the results as a string
    except Exception as e:
        return f"Error during search: {str(e)}"

realtime_search_tool = Tool(
    name="realtime_search",
    func=realtime_search,
    description="This tool performs real-time searches using Tavily and returns relevant results."
)

# Add tools to the tools list
tools = [realtime_search_tool]

# Initialize the LLM
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", api_key=GOOGLE_API_KEY)
llm_with_tools = llm.bind_tools(tools)

# Define system message
sys_msg = (
    "you are a realtime searching llm you are expert to use your tool for realtime searching when user asked anything You can use this tool to search and give response to me make sure dont give object to user give response in readable form:\n"
    "1. `realtime_search`: Search for real-time information on the web.\n"
)

# Define the assistant function
def assistant(state: MessagesState) -> MessagesState:
    """
    Assistant function that handles LLM responses.
    """
    return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}

# Create the graph
builder: StateGraph = StateGraph(MessagesState)

# Add nodes
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Define edges
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    tools_condition,  # If the assistant response is a tool call
)
builder.add_edge("tools", "assistant")

# Compile the graph
react_graph: CompiledStateGraph = builder.compile()

# User interaction
messages = [HumanMessage(content=input("User: "))]
messages = react_graph.invoke({"messages": messages})

# Print the output messages
for m in messages['messages']:
    print(m.content)


User: who is the president of US
who is the president of US

Search Results: {'query': 'president of US', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'President of the United States - Simple English Wikipedia, the free ...', 'url': 'https://simple.wikipedia.org/wiki/President_of_the_United_States', 'content': 'The president is also the head of the executive branch of the federal government of the United States and is the chairman of the presidential cabinet.[10]\nJoe Biden is the 46th and current president of the United States, in office since January 2021.[11]\nEligibility and requirements[change | change source]\nArticle II, Section 1, Clause 5 of the constitution states for a person to serve as president must:\nElection process and presidential terms[change | change source]\nThe president is elected by the people through the Electoral College to a four-year term, along with the vice presidential candidate or the incumbent vice president of the Un

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults


# Initialize Tavily search tool
tavily_search = TavilySearchResults(max_results = 3)

# Maintain chat history
chat_history = []

def stream_graph_updates(user_input):
    """A placeholder function for processing graph updates."""
    print(f"Processing graph updates for: {user_input}")


print("Hello there! I am an browsing agent. Please give me a prompt, and I will bring you the answer by searching on Google")
print("Type 'exit' , 'quit' , 'q' , 'bye' for ending")


def main():
  """ Get realtime information using tavily search """

  while True:


    try:
        # Prompt the user for input
        print("\n")
        user_input = input("User : ").strip()

        # Exit condition
        if user_input.lower() in ["quit", "exit", "q","bye"]:
            print("Bye! Have a great day!")
            break

        # Handle empty input
        if not user_input:
            print("No prompt found. Please type something.")
            continue

        # Add user input to chat history
        chat_history.append({"role": "user", "content": user_input})

        # Call the placeholder function (replace as needed)
        try:
            stream_graph_updates(user_input)
        except NameError:
            # Handle missing function gracefully
            print("stream_graph_updates is not implemented. Skipping.")

        # Perform a Tavily search if the function is not implemented
        search_docs = tavily_search.invoke(user_input)

        # Add Tavily response to chat history
        chat_history.append({"role": "assistant", "content": str(search_docs)})

        # Display results from Tavily search
        print("Search Results:")
        for i, doc in enumerate(search_docs, start=1):
            print(f"\nResult {i}:")
            print(f"Here is the {i} documentaion from web search you can get your answer from here:")
            print(f"Documentation: {doc.get('content', 'N/A')}")
            print(f"URL: {doc.get('url', 'N/A')}")

    except Exception as e:
        print(f"An error occurred: {e}")
        break

    print("\nChat History:")
    for message in chat_history:
      role = message["role"]
      content = message["content"]
      print(f"{role.capitalize()}: {content}")



main()

Hello there! I am an browsing agent. Please give me a prompt, and I will bring you the answer by searching on Google
Type 'exit' , 'quit' , 'q' , 'bye' for ending


User : who is the president of US
Processing graph updates for: who is the president of US
Search Results:

Result 1:
Here is the 1 documentaion from web search you can get your answer from here:
Documentation: The president is also the head of the executive branch of the federal government of the United States and is the chairman of the presidential cabinet.[10]
Joe Biden is the 46th and current president of the United States, in office since January 2021.[11]
Eligibility and requirements[change | change source]
Article II, Section 1, Clause 5 of the constitution states for a person to serve as president must:
Election process and presidential terms[change | change source]
The president is elected by the people through the Electoral College to a four-year term, along with the vice presidential candidate or the incumbent vi

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langgraph.graph import MessagesState, START, StateGraph
from langgraph.prebuilt import ToolNode
from langchain.agents import Tool

# Real-time search tool definition
def realtime_search(query: str) -> str:
    """
    Perform real-time searching using Tavily.
    """
    from tavily import TavilyClient
    client = TavilyClient()  # Replace with your API key
    try:
        results = client.search(query)
        if results["results"]:
            title = results["results"][0]["title"]
            snippet = results["results"][0].get("snippet", "No additional details available.")
            return f"Result: {title}\nDetails: {snippet}"
        return "No relevant results found for your query."
    except Exception as e:
        return f"Error during search: {str(e)}"

# Define the tool
realtime_search_tool = Tool(
    name="realtime_search",
    func=realtime_search,
    description="Fetch real-time information from Tavily based on the query."
)

# Initialize the tools
tools = [realtime_search_tool]

# Initialize the LLM
 # GOOGLE_API_KEY = "your_google_api_key_here" Replace with your actual API key
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", api_key=GOOGLE_API_KEY)
llm_with_tools = llm.bind_tools(tools)

# System message
sys_msg = (
    "You are a real-time search assistant capable of using tools like 'realtime_search' "
    "for queries requiring post-2020 or current data. Analyze the user's query:\n"
    "- If the query is about historical or general knowledge (before 2020), answer using your training.\n"
    "- If the query is about current events or specific post-2020 knowledge, use the tool to fetch real-time data.\n"
    "Always provide a clear and concise response."
)

# Define the assistant function
def assistant(state: MessagesState) -> MessagesState:
    """
    Assistant function that handles LLM responses and tool invocation.
    """
    # Combine system message with user messages
    messages = [SystemMessage(content=sys_msg)] + state["messages"]
    print("Debug: Sending messages to LLM:", messages)  # Debugging output

    # Get LLM's response
    llm_response = llm_with_tools.invoke(messages)
    print("Debug: Received LLM response:", llm_response)  # Debugging output

    # Check if the LLM suggests a tool call
    if "function_call" in llm_response.additional_kwargs:
        tool_name = llm_response.additional_kwargs["function_call"]["name"]
        tool_args = llm_response.additional_kwargs["function_call"]["arguments"]
        print(f"Debug: Tool {tool_name} invoked with arguments: {tool_args}")  # Debugging output

        # Execute the tool and get the output
        if tool_name == "realtime_search":
            query = eval(tool_args)["__arg1"]
            tool_output = realtime_search_tool.func(query)
            return {"messages": [AIMessage(content=tool_output)]}

    # Default behavior: return LLM response
    return {"messages": [llm_response]}

# Create the state graph
builder: StateGraph = StateGraph(MessagesState)
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))
builder.add_edge(START, "assistant")
builder.add_edge("tools", "assistant")  # Invoke tools and loop back to assistant
react_graph: CompiledStateGraph = builder.compile()

# Interactive user session
def run_chat():
    print("Welcome to the Real-Time Search Chatbot! Type 'exit' to quit.")
    while True:
        user_input = input("User: ")
        if user_input.lower() in {"exit", "quit"}:
            print("Exiting the chat. Goodbye!")
            break
        # Pass user input to the chatbot
        messages = [HumanMessage(content=user_input)]
        response = react_graph.invoke({"messages": messages})
        # Print the chatbot's response
        for msg in response["messages"]:
            print(msg.content)

# Run the chatbot
if __name__ == "__main__":
    run_chat()


Welcome to the Real-Time Search Chatbot! Type 'exit' to quit.
User: hey can you tell me who is the prime minister of pakistan in 2024
Debug: Sending messages to LLM: [SystemMessage(content="You are a real-time search assistant capable of using tools like 'realtime_search' for queries requiring post-2020 or current data. Analyze the user's query:\n- If the query is about historical or general knowledge (before 2020), answer using your training.\n- If the query is about current events or specific post-2020 knowledge, use the tool to fetch real-time data.\nAlways provide a clear and concise response.", additional_kwargs={}, response_metadata={}), HumanMessage(content='hey can you tell me who is the prime minister of pakistan in 2024', additional_kwargs={}, response_metadata={}, id='c54c9814-f9fe-4c4f-a70c-18aae5a6a472')]
Debug: Received LLM response: content='' additional_kwargs={'function_call': {'name': 'realtime_search', 'arguments': '{"__arg1": "prime minister of pakistan in 2024"}'}}

In [ ]:
from datetime import datetime
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langgraph.graph import MessagesState, START, StateGraph
from langgraph.prebuilt import ToolNode
from langchain.agents import Tool

# Real-time search tool definition
def realtime_search(query: str) -> str:
    """
    Perform real-time searching using Tavily.
    """
    from tavily import TavilyClient
    client = TavilyClient()  # Replace with your API key
    try:
        results = client.search(query)
        if results["results"]:
            title = results["results"][0]["title"]
            snippet = results["results"][0].get("snippet", "No additional details available.")
            return f"Result: {title}\nDetails: {snippet}"
        return "No relevant results found for your query."
    except Exception as e:
        return f"Error during search: {str(e)}"

# Define the tool
realtime_search_tool = Tool(
    name="realtime_search",
    func=realtime_search,
    description="Fetch real-time information from Tavily based on the query."
)

# Initialize the tools
tools = [realtime_search_tool]

# Initialize the LLM
 # GOOGLE_API_KEY = "your_google_api_key_here" Replace with your actual API key
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", api_key=GOOGLE_API_KEY)
llm_with_tools = llm.bind_tools(tools)

# System message
sys_msg = (
    "You are a real-time search assistant capable of using tools like 'realtime_search' "
    "for queries requiring post-2020 or current data. Analyze the user's query:\n"
    "- If the query is about historical or general knowledge (before 2020), answer using your training.\n"
    "- If the query is about current events or specific post-2020 knowledge, use the tool to fetch real-time data.\n"
    "- If the query is about the current date, provide it dynamically using Python.\n"
    "Always provide a clear and concise response."
)

# Define the assistant function
def assistant(state: MessagesState) -> MessagesState:
    """
    Assistant function that handles LLM responses and tool invocation.
    """
    # Combine system message with user messages
    messages = [SystemMessage(content=sys_msg)] + state["messages"]
    print("Debug: Sending messages to LLM:", messages)  # Debugging output

    # Get LLM's response
    llm_response = llm_with_tools.invoke(messages)
    print("Debug: Received LLM response:", llm_response)  # Debugging output

    # Check if the LLM suggests a tool call
    if "function_call" in llm_response.additional_kwargs:
        tool_name = llm_response.additional_kwargs["function_call"]["name"]
        tool_args = llm_response.additional_kwargs["function_call"]["arguments"]
        print(f"Debug: Tool {tool_name} invoked with arguments: {tool_args}")  # Debugging output

        # Execute the tool and get the output
        if tool_name == "realtime_search":
            query = eval(tool_args)["__arg1"]
            tool_output = realtime_search_tool.func(query)
            return {"messages": [AIMessage(content=tool_output)]}

    # Handle date query
    user_query = state["messages"][-1].content.lower()
    if "date" in user_query:
        current_date = datetime.now().strftime("%Y-%m-%d")
        return {"messages": [AIMessage(content=f"Today's date is {current_date}.")]}

    # Default behavior: return LLM response
    return {"messages": [llm_response]}

# Create the state graph
builder: StateGraph = StateGraph(MessagesState)
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))
builder.add_edge(START, "assistant")
builder.add_edge("tools", "assistant")  # Invoke tools and loop back to assistant
react_graph: CompiledStateGraph = builder.compile()

# Interactive user session
def run_chat():
    print("Welcome to the Real-Time Search Chatbot! Type 'exit' to quit.")
    while True:
        user_input = input("User: ")
        if user_input.lower() in {"exit", "quit"}:
            print("Exiting the chat. Goodbye!")
            break
        # Pass user input to the chatbot
        messages = [HumanMessage(content=user_input)]
        response = react_graph.invoke({"messages": messages})
        # Print the chatbot's response
        for msg in response["messages"]:
            print(msg.content)

# Run the chatbot
if __name__ == "__main__":
    run_chat()


Welcome to the Real-Time Search Chatbot! Type 'exit' to quit.
User: price of dollar
Debug: Sending messages to LLM: [SystemMessage(content="You are a real-time search assistant capable of using tools like 'realtime_search' for queries requiring post-2020 or current data. Analyze the user's query:\n- If the query is about historical or general knowledge (before 2020), answer using your training.\n- If the query is about current events or specific post-2020 knowledge, use the tool to fetch real-time data.\n- If the query is about the current date, provide it dynamically using Python.\nAlways provide a clear and concise response.", additional_kwargs={}, response_metadata={}), HumanMessage(content='price of dollar', additional_kwargs={}, response_metadata={}, id='3b5d36cc-839b-4aa3-9c5b-13d9e1849cb4')]
Debug: Received LLM response: content='To answer your question accurately, I need to know which currency you\'d like to compare the dollar to.  Please specify the currency (e.g., "price of d